In [1]:
from datetime import datetime, timedelta
import glob
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time
from datetime import datetime
import dateparser
from urllib.parse import urlparse, parse_qs
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [182]:
today = datetime.today()
today_string = today.strftime('%Y-%m-%d')

In [183]:
chromedriver_path = "C:/chromedriver-win64/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [184]:
web_link = "https://www.glassdoor.com/Job/indonesia-data-analyst-jobs-SRCH_IL.0,9_IN113_KO10,22.htm"

# Navigate to the webpage containing the element
driver.get(web_link)

# Refresh the webpage
driver.refresh()

# Wait for a few seconds to see the effect
time.sleep(2)

driver.maximize_window()

In [185]:
def jobid_getter(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Get the currentJobId
    current_job_id = query_params.get('jobId', [None])[0]
    return current_job_id

def prepend_if_not_com(url):
    # Check if the URL starts with any domain ending in '.com'
    if url.split('/')[0].endswith('.com'):
        return url
    else:
        return "glassdoor.com" + url

In [186]:
job_db = pd.DataFrame(columns=[
    'company'
    , 'role'
    , 'location'
    , 'post_duration'
    , 'total_application'
    , 'job_description'
    , 'level'
    , 'employment_status'
    , 'job_function'
    , 'industries'
    , 'url'
    , 'job_id'
    , 'salary'
    , 'company_total_employees'
    , 'company_rating'
    , 'company_total_review'
])

In [187]:
auto_scroll = True
retrial_view = 0
unchanged_count = 0
unchanged_count_by_button = 0
is_button_pressed = False

while auto_scroll:
    scroll_distance = 250
    driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

    try:
        button = driver.find_elements(By.XPATH, "//button[@data-test='load-more']")
        button[0].click()
        print('click button bro')
        is_button_pressed = True
        time.sleep(2)
    except:
        print(f'Gabisa click button')
        is_button_pressed = False
        retrial_view += 1
        
        try:
            modals_window = driver.find_elements(By.XPATH, "//div[@data-test='authModalContainerV2']")[0]
        except:
            pass
        
        try:
            modals_window.find_elements(By.XPATH, "//button[@class='CloseButton']")[0].click()
        except:
            print('Tidak muncul modals')
        
        if retrial_view == 3:
            auto_scroll = False
            
print('Selesai-----------')

click button bro
Gabisa click button
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
click button bro
Gabisa click button
Tidak muncul modals
Gabisa click button
Tidak muncul modals
Selesai-----------


In [188]:
list_jobs_box = driver.find_elements(By.XPATH, "//ul[@aria-label='Jobs List']")[0]
jobs = list_jobs_box.find_elements(By.XPATH, ".//li[contains(@class, 'JobsList_jobListItem__wjTHv')]")[:-1]

In [189]:
desc = driver.find_elements(By.XPATH, ".//div[contains(@class, 'JobDetails_jobDescription__uW_fK')]")
desc

[<selenium.webdriver.remote.webelement.WebElement (session="852495d6a614b003c8cee5cc9a87f806", element="f.CB5813476835C32A4A7FB64946F4E1F1.d.DA167F2FC01562595FA7FFDA9466CF34.e.550")>]

In [192]:
for ind, i_job in enumerate(jobs[387:]):

    i_job.click()
    time.sleep(0.6)
    
    job_post_duration = i_job.find_elements(By.XPATH, ".//div[@class='JobCard_listingAge__Ny_nG']")[0].text
    
    try:
        button = driver.find_elements(By.XPATH, "//button[.//span[contains(text(), 'Show more')]]")[0]
        button.click()
    except:
        pass
    
    try:
        job_description = driver.find_elements(By.XPATH, ".//div[contains(@class, 'JobDetails_jobDescription__uW_fK')]")[0].text
    except:
        job_description = np.nan
    
    job_id = i_job.get_attribute("data-jobid")
    job_role = driver.find_elements(By.XPATH, ".//h1[@class='heading_Heading__BqX5J heading_Level1__soLZs']")[0].text
    job_company = driver.find_elements(By.XPATH, ".//h4[@class='heading_Heading__BqX5J heading_Subhead__Ip1aW']")[0].text
    
    try:
        company_rating = driver.find_elements(By.XPATH, ".//div[@class='EmployerProfile_ratingContainer__ul0Ef']")[0].text
    except:
        company_rating = np.nan

    try:
        wadah_size = driver.find_elements(By.XPATH, "//div[.//div[contains(text(), 'Employees') and @class='JobDetails_overviewItemValue__xn8EF'] and @class='JobDetails_overviewItem__cAsry']")[0]
        total_employee = wadah_size.find_elements(By.XPATH, ".//div[@class='JobDetails_overviewItemValue__xn8EF']")[0].text
    except:
        total_employee = np.nan
    
    try:
        wadah_industry = driver.find_elements(By.XPATH, "//div[.//span[contains(text(), 'Industry') and @class='JobDetails_overviewItemLabel__KjFln'] and @class='JobDetails_overviewItem__cAsry']")[0]
        job_industries = wadah_industry.find_elements(By.XPATH, ".//div[@class='JobDetails_overviewItemValue__xn8EF']")[0].text
    except:
        job_industries = np.nan

    job_location = driver.find_elements(By.XPATH, ".//div[@class='JobDetails_location__mSg5h']")[0].text
    
    try:
        elemen_apply = driver.find_elements(By.XPATH, "//button[@data-test='applyButton' or @data-test='easyApply']")[0]
        app_type = elemen_apply.get_attribute("data-test")

        if app_type == 'easyApply':
            apply_container = driver.find_elements(By.XPATH, "//div[.//button[@data-test='applyButton' or @data-test='easyApply'] and @class='JobDetails_applyButtonContainer__L36Bs']")[0]
            job_url = apply_container.find_elements(By.XPATH, ".//span[@class='indeed-apply-widget indeed-apply-status-not-applied']")[0].get_attribute("data-indeed-apply-joburl")

        elif app_type == 'applyButton':
            cek = elemen_apply.get_attribute("data-job-url")
            job_url = prepend_if_not_com(cek)
    except:
        job_url = np.nan
    
    job_db.loc[i_job, 'company'] = job_company
    job_db.loc[i_job, 'role'] = job_role
    job_db.loc[i_job, 'location'] = job_location
    job_db.loc[i_job, 'post_duration'] = job_post_duration
    job_db.loc[i_job, 'total_application'] = np.nan
    job_db.loc[i_job, 'level'] = np.nan
    job_db.loc[i_job, 'employment_status'] = np.nan
    job_db.loc[i_job, 'job_function'] = np.nan
    job_db.loc[i_job, 'industries'] = job_industries
    job_db.loc[i_job, 'job_description'] = job_description
    job_db.loc[i_job, 'url'] = job_url
    job_db.loc[i_job, 'job_id'] = job_id
    
    print(f'job ke {ind+388}/{len(jobs)}')

job ke 254/418
job ke 255/418
job ke 256/418
job ke 257/418
job ke 258/418
job ke 259/418
job ke 260/418
job ke 261/418
job ke 262/418
job ke 263/418
job ke 264/418
job ke 265/418
job ke 266/418
job ke 267/418
job ke 268/418
job ke 269/418
job ke 270/418
job ke 271/418
job ke 272/418
job ke 273/418
job ke 274/418
job ke 275/418
job ke 276/418
job ke 277/418
job ke 278/418
job ke 279/418
job ke 280/418
job ke 281/418
job ke 282/418
job ke 283/418
job ke 284/418
job ke 285/418
job ke 286/418
job ke 287/418
job ke 288/418
job ke 289/418
job ke 290/418
job ke 291/418
job ke 292/418
job ke 293/418
job ke 294/418
job ke 295/418
job ke 296/418
job ke 297/418
job ke 298/418
job ke 299/418
job ke 300/418
job ke 301/418
job ke 302/418
job ke 303/418
job ke 304/418
job ke 305/418
job ke 306/418
job ke 307/418
job ke 308/418
job ke 309/418
job ke 310/418
job ke 311/418
job ke 312/418
job ke 313/418
job ke 314/418
job ke 315/418
job ke 316/418
job ke 317/418
job ke 318/418
job ke 319/418
job ke 320

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.176)
Stacktrace:
	GetHandleVerifier [0x00007FF7172B1F52+60322]
	(No symbol) [0x00007FF71722CEC9]
	(No symbol) [0x00007FF7170E7EBA]
	(No symbol) [0x00007FF7170EDCEE]
	(No symbol) [0x00007FF7170F0641]
	(No symbol) [0x00007FF7170F06E0]
	(No symbol) [0x00007FF7171314FB]
	(No symbol) [0x00007FF71715C21A]
	(No symbol) [0x00007FF71712ADB6]
	(No symbol) [0x00007FF71715C430]
	(No symbol) [0x00007FF71717BC80]
	(No symbol) [0x00007FF71715BFC3]
	(No symbol) [0x00007FF717129617]
	(No symbol) [0x00007FF71712A211]
	GetHandleVerifier [0x00007FF7175C94AD+3301629]
	GetHandleVerifier [0x00007FF7176136D3+3605283]
	GetHandleVerifier [0x00007FF717609450+3563680]
	GetHandleVerifier [0x00007FF717364326+790390]
	(No symbol) [0x00007FF71723750F]
	(No symbol) [0x00007FF717233404]
	(No symbol) [0x00007FF717233592]
	(No symbol) [0x00007FF717222F9F]
	BaseThreadInitThunk [

In [ ]:
job_db_final = job_db.reset_index(drop=True)

In [ ]:
job_db_final